# Semantic Kernel 

In this code sample, you will use the [Semantic Kernel](https://aka.ms/ai-agents-beginners/semantic-kernel) AI Framework to create a basic agent. 

The goal of this sample is to show you the steps that we will later use in the additional code samples when implementing the different agentic patterns. 

## Import the Needed Python Packages 

In [3]:
import os 
from typing import Annotated
from openai import AsyncOpenAI

from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

## Creating the Client

In this sample, we will use [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) for access to the LLM. 

The `ai_model_id` is defined as `gpt-4o-mini`. Try changing the model to another model available on the GitHub Models marketplace to see the different results. 

For us to use the `Azure Inference SDK` that is used for the `base_url` for GitHub Models, we will use the `OpenAIChatCompletion` connector within Semantic Kernel. There are also other [available connectors](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) to use Semantic Kernel for other model providers.

In [4]:
import random
from typing import Annotated

# Define a sample plugin for dinner suggestions

class DinnerPlugin:
    """A list of random Italian dishes for dinner with calorie information."""

    def __init__(self):
        # List of Italian dishes with estimated calories
        self.dishes = [
            "Pizza Margherita - approx. 300 kcal/slice",
            "Spaghetti alla Carbonara - approx. 600 kcal/portion",
            "Eggplant Parmigiana - approx. 450 kcal/portion",
            "Grilled Salmon with Broccoli - approx. 500 kcal/portion",
            "Minestrone Soup - approx. 200 kcal/bowl",
            "Chicken Cacciatore - approx. 400 kcal/portion",
            "Caprese Salad - approx. 250 kcal/plate",
            "Lasagna alla Bolognese - approx. 650 kcal/portion",
            "Risotto ai Funghi - approx. 500 kcal/portion",
            "Orecchiette con Cime di Rapa - approx. 450 kcal/portion"
        ]
        # Track last suggestion to avoid repeats
        self.last_dish = None

    @kernel_function(description="Provides a random Italian dinner suggestion.")
    def get_random_dinner(self) -> Annotated[str, "Returns a random Italian dinner dish with calories."]:
        # Get available dishes excluding the last one, if possible
        available_dishes = self.dishes.copy()
        if self.last_dish and len(available_dishes) > 1:
            available_dishes.remove(self.last_dish)

        # Select a random dish
        dish = random.choice(available_dishes)

        # Update the last suggested dish
        self.last_dish = dish

        return dish


In [5]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## Creating the Agent 

Below we create the Agent called `TravelAgent`.

For this example, we are using very simple instructions. You can change these instructions to see how the agent responds differently. 

In [9]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DinnerPlugin()],
    name="DinnerSuggestionAgent",
    instructions="You are a helpful AI Agent that can help choose a random Italian dinner dish with calorie information. Explaining how to make the dish",
)

## Running the Agent

Now we can run the Agent by defining a thread of type `ChatHistoryAgentThread`.  Any required system messages are provided to the agent's invoke_stream `messages` keyword argument.

After these are defined, we create a `user_inputs` that will be what the user is sending to the agent. In this case, we have set this message to `Plan me a sunny vacation`. 

Feel free to change this message to see how the agent responds differently. 

In [10]:
async def main():
    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Plan me a dinner for tonight.",
    ]

    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. Print the response
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    # Clean up the thread
    await thread.delete() if thread else None

await main()

# User: Plan me a dinner for tonight.

# DinnerSuggestionAgent: How about making **Orecchiette con Cime di Rapa** for dinner tonight? This dish is delicious and has approximately 450 calories per portion. 

### Ingredients:
- 400g orecchiette pasta
- 500g cime di rapa (broccoli rabe)
- 3 cloves garlic, minced
- 1/4 tsp red pepper flakes (adjust to taste)
- 4 tbsp olive oil
- Salt to taste
- Grated Parmesan cheese (optional)

### Instructions:

1. **Prepare the Greens**: Start by cleaning the cime di rapa. Remove the tough stems, and rinse them thoroughly under cold water. 

2. **Cook the Pasta**: In a large pot, bring salted water to a boil and add the orecchiette. Cook according to package instructions until al dente.

3. **Blanch the Cime di Rapa**: In the last 3-4 minutes of pasta cooking time, add the cime di rapa to the pot. Let it cook with the pasta until both are tender.

4. **Sauté the Garlic**: Meanwhile, in a large skillet, heat the olive oil over medium heat. Add the minced